# Applied Data Science Capstone Week 5

## Project Setup

### Import Libraries

In [1]:
import pandas as pd
import numpy as np
import folium
import requests
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

### Read the main dataset 'complete_dataset.csv' into a DataFrame

In [2]:
complete = pd.read_csv('e:/data/complete_dataset.csv')
complete.drop('Unnamed: 0',axis=1,inplace=True)

### Create location dataset

In [3]:
location = pd.DataFrame(complete[['Borough', 'Neighborhoods', 'Latitude', 'Longitude']])
location.to_csv('e:/data/location_dataset.csv')
location.head(10)

,Borough,Neighborhoods,Latitude,Longitude
0,Central,Cantonment area,12.972442,77.580643
1,Central,Domlur,12.960992,77.638726
2,Central,Indiranagar,12.971891,77.641151
3,Central,Jeevanbheemanagar,12.962900,77.659500
4,Central,Malleswaram,13.003100,77.564300
5,Central,Pete area,12.962700,77.575800
6,Central,Rajajinagar,12.990100,77.552500
7,Central,Sadashivanagar,13.006800,77.581300
8,Central,Seshadripuram,12.993500,77.578700
9,Central,Shivajinagar,12.985700,77.605700


### Create population dataset

In [4]:
population = pd.DataFrame(complete[['Borough','Neighborhoods','Population']])
population.to_csv('e:/data/population_dataset.csv')
population.head(10)

,Borough,Neighborhoods,Population
0,Central,Cantonment area,866377
1,Central,Domlur,743186
2,Central,Indiranagar,474289
3,Central,Jeevanbheemanagar,527874
4,Central,Malleswaram,893629
5,Central,Pete area,730999
6,Central,Rajajinagar,981362
7,Central,Sadashivanagar,662625
8,Central,Seshadripuram,396862
9,Central,Shivajinagar,77836


### Create income dataset

In [5]:
income = pd.DataFrame(complete[['Borough','Neighborhoods','AverageIncome']])
income.to_csv('e:/data/income_dataset.csv')
income.head(10)

,Borough,Neighborhoods,AverageIncome
0,Central,Cantonment area,18944.099792
1,Central,Domlur,56837.022198
2,Central,Indiranagar,41991.817435
3,Central,Jeevanbheemanagar,6667.447632
4,Central,Malleswaram,53270.063892
5,Central,Pete area,50712.430215
6,Central,Rajajinagar,60967.535874
7,Central,Sadashivanagar,59943.541564
8,Central,Seshadripuram,58407.090338
9,Central,Shivajinagar,55850.962099


### Get mean latitude and longitude of Bengaluru

In [6]:
latitude = location['Latitude'].mean()
longitude = location['Longitude'].mean()
print("Latitude and Longitude of Bengaluru : ", latitude, longitude)

Latitude and Longitude of Bengaluru :  12.962339620312497 77.60175294687502


### Get unique Boroughs in Bengaluru

In [7]:
unique_boroughs = location['Borough'].unique().tolist()
unique_boroughs

['Central',
 'Eastern',
 'NorthEastern',
 'Northern',
 'SouthEastern',
 'Southern',
 'SouthernSuburbs',
 'Western']

### Create map of Bengaluru

In [40]:
borough_colors ={}
for i in unique_boroughs:
    borough_colors[i] = '#%02X%02X%02X' % tuple(np.random.choice(range(256),size=3))
map = folium.Map(location=[latitude, longitude], zoom_start=12, control_scale=True)
for lat, lng, boro, nei in zip(location['Latitude'], location['Longitude'], location['Borough'], location['Neighborhoods']):
    label_text = boro + ' - ' + nei
    label = folium.Popup(label_text, parse_html=True)
    folium.CircleMarker(
        [lat,lng],
        tooltip = label_text,
        radius = 4,
        popup = label,
        color=borough_colors[boro],
        fill=True,
        fill_color = borough_colors[boro],
        fill_opacity=0.7
    ).add_to(map)
map

## Exploring the neighbors using FourSquare API

### Get neighborhood venues and categories

In [9]:
CLIENT_ID = 'ZK40VO3EPO1ZY5C04PNYSMQH2ZFZ3U1TPL0QME3WEY553DMH'
CLIENT_SECRET = 'RQBRNE2T3OTMYY4BBJ3YWLS0P2FK413O22MQV1QHBHAD04WG'
VERSION = '20180606'
LIMIT = 150

In [10]:
def getNearbyVenues(names, boro, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, boro, lat, lng in zip(names, boro, latitudes, longitudes):
        # print("Fetching venues for : ",name)            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            boro,
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood',
                             'Borough',
                             'Neighborhood Latitude', 
                             'Neighborhood Longitude', 
                             'Venue', 
                             'Venue Latitude', 
                             'Venue Longitude', 
                             'Venue Category']
    
    return(nearby_venues)

In [11]:
venues = getNearbyVenues(names=location['Neighborhoods'],latitudes=location['Latitude'],longitudes=location['Longitude'],boro=location['Borough'])
print("Total number of venues = ", venues.shape[0])
venues.head(10)

Total number of venues =  594


,Neighborhood,Borough,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Cantonment area,Central,12.972442,77.580643,Hotel Fishland,12.975569,77.578592,Seafood Restaurant
1,Cantonment area,Central,12.972442,77.580643,Vasudev Adigas,12.973707,77.579257,Indian Restaurant
2,Cantonment area,Central,12.972442,77.580643,Adigas Hotel,12.973554,77.579161,Restaurant
3,Cantonment area,Central,12.972442,77.580643,Sapna Book House,12.976355,77.578461,Bookstore
4,Cantonment area,Central,12.972442,77.580643,Kamat Yatrinivas,12.975985,77.578125,Indian Restaurant
5,Domlur,Central,12.960992,77.638726,Lavonne,12.963909,77.638579,Café
6,Domlur,Central,12.960992,77.638726,Barbeque Nation,12.962684,77.641599,BBQ Joint
7,Domlur,Central,12.960992,77.638726,Mainland China,12.962458,77.641727,Chinese Restaurant
8,Domlur,Central,12.960992,77.638726,Domino's Pizza,12.961000,77.639000,Pizza Place
9,Domlur,Central,12.960992,77.638726,Srinidhi Sagar,12.959348,77.638387,Indian Restaurant


In [12]:
venues.groupby('Venue Category').count()['Neighborhood'].sort_values(ascending=False)
print("Total number of unique categories = ",len(venues['Venue Category'].unique()))

Total number of unique categories =  124


### Get number of venues per neighborhood

In [13]:
individual_venue_count = venues.groupby(['Borough','Neighborhood'])['Borough'].count().to_frame()
individual_venue_count

Borough
Borough         Neighborhood                 
Central         Cantonment area             5
                Domlur                     11
                Indiranagar                65
                Jeevanbheemanagar           5
                Malleswaram                 6
                Pete area                   4
                Rajajinagar                11
                Sadashivanagar             20
                Seshadripuram               8
                Shivajinagar               11
                Ulsoor                      6
                Vasanth Nagar              24
Eastern         Bellandur                  20
                CV Raman Nagar              7
                Hoodi                       5
                Krishnarajapuram            1
                Mahadevapura                5
                Marathahalli                7
                Varthur                     1
                Whitefield                 10
NorthEastern    Banaswadi                   8
                HBR Layout                  6
                Horamavu                    1
                Kammanahalli                9
                Lingarajapuram              1
                Ramamurthy Nagar            6
Northern        Hebbal                      7
                Jalahalli                   3
                Mathikere                  12
                R. T. Nagar                 8
...                                       ...
                Yeshwanthpur                9
SouthEastern    BTM Layout                 39
                Bommanahalli                6
                Bommasandra                 1
                Electronic City             6
                HSR Layout                 10
                Koramangala                 7
                Madiwala                   24
Southern        Banashankari               11
                Basavanagudi               11
                Girinagar                   2
                J. P. Nagar                29
                Jayanagar                  13
                Kumaraswamy Layout          8
                Padmanabhanagar             1
                Uttarahalli                12
SouthernSuburbs Arekere                    25
                Begur                       4
                Gottigere                   8
                Hulimavu                    6
                Kothnur                     4
Western         Basaveshwaranagar          22
                Kamakshipalya               1
                Kengeri                     6
                Mahalakshmi Layout         11
                Nagarbhavi                  7
                Nandini Layout              1
                Nayandahalli                3
                Rajarajeshwari Nagar        7
                Vijayanagar                 3

[61 rows x 1 columns]

### Explore 'Malleswaram' venues

In [14]:
venues[venues['Neighborhood']=='Malleswaram']

,Neighborhood,Borough,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
86,Malleswaram,Central,13.0031,77.5643,Raghavendra Stores,13.000799,77.563924,Breakfast Spot
87,Malleswaram,Central,13.0031,77.5643,Sai Ram Chats,13.006615,77.564158,Indian Restaurant
88,Malleswaram,Central,13.0031,77.5643,Naturals Ice Cream,13.006587,77.564354,Ice Cream Shop
89,Malleswaram,Central,13.0031,77.5643,Bun World,13.007511,77.563738,Bakery
90,Malleswaram,Central,13.0031,77.5643,Amrith Ice creams,13.001860,77.567340,Ice Cream Shop
91,Malleswaram,Central,13.0031,77.5643,baskin robbins,13.007388,77.563982,Ice Cream Shop


### Get top 5 venues for each neighborhood

In [15]:
venues_onehot = pd.get_dummies(venues[['Venue Category']])
venues_onehot['Neighborhood'] = venues['Neighborhood']
venues_grouped = venues_onehot.groupby('Neighborhood').mean().reset_index()
venues_grouped

,Neighborhood,Venue Category_Andhra Restaurant,Venue Category_Art Museum,Venue Category_Asian Restaurant,Venue Category_Athletics & Sports,Venue Category_Auto Garage,Venue Category_Auto Workshop,Venue Category_BBQ Joint,Venue Category_Badminton Court,Venue Category_Bakery,...,Venue Category_Supermarket,Venue Category_Tea Room,Venue Category_Tex-Mex Restaurant,Venue Category_Toll Plaza,Venue Category_Train Station,Venue Category_Udupi Restaurant,Venue Category_Vegetarian / Vegan Restaurant,Venue Category_Vietnamese Restaurant,Venue Category_Women's Store,Venue Category_Yoga Studio
0,Arekere,0.000000,0.000000,0.000000,0.040000,0.000000,0.000000,0.000000,0.040000,0.000000,...,0.040000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00
1,BTM Layout,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.076923,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.051282,0.000000,0.00,0.00
2,Banashankari,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00
3,Banaswadi,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.125000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.125000,0.000000,0.00,0.00
4,Basavanagudi,0.000000,0.000000,0.000000,0.090909,0.000000,0.000000,0.000000,0.000000,0.090909,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00
5,Basaveshwaranagar,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.045455,...,0.045455,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00
6,Begur,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00
7,Bellandur,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00
8,Bommanahalli,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.166667,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00
9,Bommasandra,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00


In [16]:
number_of_top_venues = 5
for hood in venues_grouped['Neighborhood']:
    print('---------',hood,'---------')
    temp = venues_grouped[venues_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['Venue','Frequency']
    temp = temp.iloc[1:]
    temp['Frequency'] = temp['Frequency'].astype(float)
    temp = temp.round({'Frequency': 2})
    print(temp.sort_values('Frequency', ascending=False).reset_index(drop=True).head(number_of_top_venues))
    print('\n')

--------- Arekere ---------
                                Venue  Frequency
0    Venue Category_Indian Restaurant       0.20
1  Venue Category_Sporting Goods Shop       0.16
2       Venue Category_Ice Cream Shop       0.04
3          Venue Category_Supermarket       0.04
4         Venue Category_Dessert Shop       0.04


--------- BTM Layout ---------
                               Venue  Frequency
0   Venue Category_Indian Restaurant       0.21
1      Venue Category_Ice Cream Shop       0.08
2              Venue Category_Bakery       0.08
3  Venue Category_Chinese Restaurant       0.08
4         Venue Category_Snack Place       0.08


--------- Banashankari ---------
                              Venue  Frequency
0  Venue Category_Indian Restaurant       0.18
1               Venue Category_Café       0.18
2        Venue Category_Pizza Place       0.09
3         Venue Category_Shoe Store       0.09
4     Venue Category_Breakfast Spot       0.09


--------- Banaswadi ---------
        

                                      Venue  Frequency
0       Venue Category_Fast Food Restaurant        0.4
1                   Venue Category_Bus Line        0.2
2  Venue Category_Middle Eastern Restaurant        0.2
3                Venue Category_Pizza Place        0.2
4            Venue Category_Paintball Field        0.0


--------- Kamakshipalya ---------
                                    Venue  Frequency
0  Venue Category_South Indian Restaurant        1.0
1       Venue Category_Mexican Restaurant        0.0
2                     Venue Category_Park        0.0
3          Venue Category_Paintball Field        0.0
4             Venue Category_Outlet Store        0.0


--------- Kammanahalli ---------
                                 Venue  Frequency
0  Venue Category_Fast Food Restaurant       0.22
1     Venue Category_Korean Restaurant       0.11
2           Venue Category_Snack Place       0.11
3    Venue Category_Chinese Restaurant       0.11
4    Venue Category_Falafel Res

                                    Venue  Frequency
0        Venue Category_Indian Restaurant       0.36
1               Venue Category_Donut Shop       0.09
2     Venue Category_Fast Food Restaurant       0.09
3  Venue Category_South Indian Restaurant       0.09
4                   Venue Category_Market       0.09


--------- Ulsoor ---------
                          Venue  Frequency
0           Venue Category_Café       0.33
1    Venue Category_Bridal Shop       0.17
2  Venue Category_Auto Workshop       0.17
3         Venue Category_Bakery       0.17
4   Venue Category_Burger Joint       0.17


--------- Uttarahalli ---------
                              Venue  Frequency
0  Venue Category_Indian Restaurant       0.33
1             Venue Category_Bakery       0.17
2        Venue Category_Pizza Place       0.08
3         Venue Category_Sports Bar       0.08
4                Venue Category_Gym       0.08


--------- Varthur ---------
                               Venue  Frequency
0

### Get most common venues

In [17]:
def return_most_common_venues(row, number_of_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:number_of_top_venues]

In [18]:
number_of_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(number_of_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = venues_grouped['Neighborhood']

for ind in np.arange(venues_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(venues_grouped.iloc[ind, :], number_of_top_venues)

neighborhoods_venues_sorted.shape

(61, 11)

In [19]:
neighborhoods_venues_sorted.head(10)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Arekere,Venue Category_Indian Restaurant,Venue Category_Sporting Goods Shop,Venue Category_Mughlai Restaurant,Venue Category_Liquor Store,Venue Category_Bus Line,Venue Category_Burger Joint,Venue Category_Neighborhood,Venue Category_Breakfast Spot,Venue Category_Pizza Place,Venue Category_Bar
1,BTM Layout,Venue Category_Indian Restaurant,Venue Category_Chinese Restaurant,Venue Category_Ice Cream Shop,Venue Category_Bakery,Venue Category_Snack Place,Venue Category_Vegetarian / Vegan Restaurant,Venue Category_Sandwich Place,Venue Category_Coffee Shop,Venue Category_Pizza Place,Venue Category_Department Store
2,Banashankari,Venue Category_Indian Restaurant,Venue Category_Café,Venue Category_Breakfast Spot,Venue Category_Fried Chicken Joint,Venue Category_North Indian Restaurant,Venue Category_Pizza Place,Venue Category_Clothing Store,Venue Category_Shoe Store,Venue Category_Sporting Goods Shop,Venue Category_Fish & Chips Shop
3,Banaswadi,Venue Category_Indian Restaurant,Venue Category_Bakery,Venue Category_Bookstore,Venue Category_South Indian Restaurant,Venue Category_Grocery Store,Venue Category_Market,Venue Category_Vegetarian / Vegan Restaurant,Venue Category_Donut Shop,Venue Category_Eastern European Restaurant,Venue Category_Electronics Store
4,Basavanagudi,Venue Category_Indian Restaurant,Venue Category_Café,Venue Category_Hookah Bar,Venue Category_Restaurant,Venue Category_Metro Station,Venue Category_Bakery,Venue Category_Athletics & Sports,Venue Category_Eastern European Restaurant,Venue Category_Electronics Store,Venue Category_Event Space
5,Basaveshwaranagar,Venue Category_Ice Cream Shop,Venue Category_Fast Food Restaurant,Venue Category_Indian Restaurant,Venue Category_Gastropub,Venue Category_Sporting Goods Shop,Venue Category_Food Truck,Venue Category_Clothing Store,Venue Category_Snack Place,Venue Category_Pizza Place,Venue Category_Karnataka Restaurant
6,Begur,Venue Category_Food Court,Venue Category_North Indian Restaurant,Venue Category_Clothing Store,Venue Category_Café,Venue Category_Department Store,Venue Category_Dim Sum Restaurant,Venue Category_Diner,Venue Category_Donut Shop,Venue Category_Eastern European Restaurant,Venue Category_Electronics Store
7,Bellandur,Venue Category_Indian Restaurant,Venue Category_Fast Food Restaurant,Venue Category_Café,Venue Category_Pizza Place,Venue Category_Shopping Mall,Venue Category_Punjabi Restaurant,Venue Category_Dessert Shop,Venue Category_Moving Target,Venue Category_Kerala Restaurant,Venue Category_Burger Joint
8,Bommanahalli,Venue Category_Department Store,Venue Category_Bakery,Venue Category_Indian Restaurant,Venue Category_Athletics & Sports,Venue Category_Fast Food Restaurant,Venue Category_Event Space,Venue Category_Food & Drink Shop,Venue Category_Flea Market,Venue Category_Fish & Chips Shop,Venue Category_Falafel Restaurant
9,Bommasandra,Venue Category_Indian Restaurant,Venue Category_Yoga Studio,Venue Category_Food Truck,Venue Category_Dessert Shop,Venue Category_Dim Sum Restaurant,Venue Category_Diner,Venue Category_Donut Shop,Venue Category_Eastern European Restaurant,Venue Category_Electronics Store,Venue Category_Event Space


### Cluster the neighborhoods

In [20]:
kmeans = KMeans(n_clusters=5,random_state=0)
kmeans.fit(venues_grouped.drop('Neighborhood',axis=1))

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=5, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=0, tol=0.0001, verbose=0)

In [21]:
venues_grouped['Cluster'] = kmeans.labels_
venues_grouped.groupby('Cluster')['Neighborhood'].count()

Cluster
0    37
1     1
2    21
3     1
4     1
Name: Neighborhood, dtype: int64

In [22]:
neighborhoods_venues_sorted = neighborhoods_venues_sorted.merge(venues_grouped,on='Neighborhood')
neighborhoods_venues_sorted.head(4)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,Venue Category_Tea Room,Venue Category_Tex-Mex Restaurant,Venue Category_Toll Plaza,Venue Category_Train Station,Venue Category_Udupi Restaurant,Venue Category_Vegetarian / Vegan Restaurant,Venue Category_Vietnamese Restaurant,Venue Category_Women's Store,Venue Category_Yoga Studio,Cluster
0,Arekere,Venue Category_Indian Restaurant,Venue Category_Sporting Goods Shop,Venue Category_Mughlai Restaurant,Venue Category_Liquor Store,Venue Category_Bus Line,Venue Category_Burger Joint,Venue Category_Neighborhood,Venue Category_Breakfast Spot,Venue Category_Pizza Place,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0
1,BTM Layout,Venue Category_Indian Restaurant,Venue Category_Chinese Restaurant,Venue Category_Ice Cream Shop,Venue Category_Bakery,Venue Category_Snack Place,Venue Category_Vegetarian / Vegan Restaurant,Venue Category_Sandwich Place,Venue Category_Coffee Shop,Venue Category_Pizza Place,...,0.0,0.0,0.0,0.0,0.0,0.051282,0.0,0.0,0.0,0
2,Banashankari,Venue Category_Indian Restaurant,Venue Category_Café,Venue Category_Breakfast Spot,Venue Category_Fried Chicken Joint,Venue Category_North Indian Restaurant,Venue Category_Pizza Place,Venue Category_Clothing Store,Venue Category_Shoe Store,Venue Category_Sporting Goods Shop,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0
3,Banaswadi,Venue Category_Indian Restaurant,Venue Category_Bakery,Venue Category_Bookstore,Venue Category_South Indian Restaurant,Venue Category_Grocery Store,Venue Category_Market,Venue Category_Vegetarian / Vegan Restaurant,Venue Category_Donut Shop,Venue Category_Eastern European Restaurant,...,0.0,0.0,0.0,0.0,0.0,0.125000,0.0,0.0,0.0,2


In [23]:
neighborhoods_venues_sorted.columns
neighborhoods_venues_sorted = neighborhoods_venues_sorted.merge(venues,on='Neighborhood')

In [24]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
x = np.arange(6)
ys = [i + x + (i*x)**2 for i in range(6)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(neighborhoods_venues_sorted['Neighborhood Latitude'], neighborhoods_venues_sorted['Neighborhood Longitude'], neighborhoods_venues_sorted['Neighborhood'], neighborhoods_venues_sorted['Cluster']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
map_clusters

### Recommend similar locations as required by the user (based on population and income)

### Initial setup

In [25]:
population['Normalized_population'] = population['Population']/population['Population'].max(axis=0)
population.head()

,Borough,Neighborhoods,Population,Normalized_population
0,Central,Cantonment area,866377,0.880810
1,Central,Domlur,743186,0.755567
2,Central,Indiranagar,474289,0.482190
3,Central,Jeevanbheemanagar,527874,0.536668
4,Central,Malleswaram,893629,0.908516


In [26]:
income['Normalized_income'] = income['AverageIncome']/income['AverageIncome'].max(axis=0)
income.head()

,Borough,Neighborhoods,AverageIncome,Normalized_income
0,Central,Cantonment area,18944.099792,0.293051
1,Central,Domlur,56837.022198,0.879225
2,Central,Indiranagar,41991.817435,0.649581
3,Central,Jeevanbheemanagar,6667.447632,0.103140
4,Central,Malleswaram,53270.063892,0.824047


### Recommending Veg Restaurants

In [27]:
veg = venues_onehot.groupby(['Neighborhood']).sum().reset_index()
veg.head()

,Neighborhood,Venue Category_Andhra Restaurant,Venue Category_Art Museum,Venue Category_Asian Restaurant,Venue Category_Athletics & Sports,Venue Category_Auto Garage,Venue Category_Auto Workshop,Venue Category_BBQ Joint,Venue Category_Badminton Court,Venue Category_Bakery,...,Venue Category_Supermarket,Venue Category_Tea Room,Venue Category_Tex-Mex Restaurant,Venue Category_Toll Plaza,Venue Category_Train Station,Venue Category_Udupi Restaurant,Venue Category_Vegetarian / Vegan Restaurant,Venue Category_Vietnamese Restaurant,Venue Category_Women's Store,Venue Category_Yoga Studio
0,Arekere,0,0,0,1,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
1,BTM Layout,0,0,0,0,0,0,0,0,3,...,0,0,0,0,0,0,2,0,0,0
2,Banashankari,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Banaswadi,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
4,Basavanagudi,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [28]:
veg.rename(columns={'Neighborhood':'Neighborhoods'},inplace=True)
veg.rename(columns={'Venue Category_Vegetarian / Vegan Restaurant':'NumberOfVegRestaurants'},inplace=True)
veg = veg[['Neighborhoods','NumberOfVegRestaurants']]
veg.head(10)

,Neighborhoods,NumberOfVegRestaurants
0,Arekere,0
1,BTM Layout,2
2,Banashankari,0
3,Banaswadi,1
4,Basavanagudi,0
5,Basaveshwaranagar,0
6,Begur,0
7,Bellandur,0
8,Bommanahalli,0
9,Bommasandra,0


In [29]:
veg['NumberOfNonVegRestaurants'] = 1-(veg['NumberOfVegRestaurants']/veg['NumberOfVegRestaurants'].max(axis=0))
veg.head(10)

,Neighborhoods,NumberOfVegRestaurants,NumberOfNonVegRestaurants
0,Arekere,0,1.0
1,BTM Layout,2,0.0
2,Banashankari,0,1.0
3,Banaswadi,1,0.5
4,Basavanagudi,0,1.0
5,Basaveshwaranagar,0,1.0
6,Begur,0,1.0
7,Bellandur,0,1.0
8,Bommanahalli,0,1.0
9,Bommasandra,0,1.0


### Construct a target neighbohood for "Malleswaram"

In [30]:
target_cluster_dataframe = neighborhoods_venues_sorted.loc[neighborhoods_venues_sorted['Neighborhood']=='Malleswaram']
target_cluster_dataframe.head(10)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,Venue Category_Women's Store,Venue Category_Yoga Studio,Cluster,Borough,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
413,Malleswaram,Venue Category_Ice Cream Shop,Venue Category_Breakfast Spot,Venue Category_Indian Restaurant,Venue Category_Bakery,Venue Category_Falafel Restaurant,Venue Category_Food Court,Venue Category_Food & Drink Shop,Venue Category_Flea Market,Venue Category_Fish & Chips Shop,...,0.0,0.0,0,Central,13.0031,77.5643,Raghavendra Stores,13.000799,77.563924,Breakfast Spot
414,Malleswaram,Venue Category_Ice Cream Shop,Venue Category_Breakfast Spot,Venue Category_Indian Restaurant,Venue Category_Bakery,Venue Category_Falafel Restaurant,Venue Category_Food Court,Venue Category_Food & Drink Shop,Venue Category_Flea Market,Venue Category_Fish & Chips Shop,...,0.0,0.0,0,Central,13.0031,77.5643,Sai Ram Chats,13.006615,77.564158,Indian Restaurant
415,Malleswaram,Venue Category_Ice Cream Shop,Venue Category_Breakfast Spot,Venue Category_Indian Restaurant,Venue Category_Bakery,Venue Category_Falafel Restaurant,Venue Category_Food Court,Venue Category_Food & Drink Shop,Venue Category_Flea Market,Venue Category_Fish & Chips Shop,...,0.0,0.0,0,Central,13.0031,77.5643,Naturals Ice Cream,13.006587,77.564354,Ice Cream Shop
416,Malleswaram,Venue Category_Ice Cream Shop,Venue Category_Breakfast Spot,Venue Category_Indian Restaurant,Venue Category_Bakery,Venue Category_Falafel Restaurant,Venue Category_Food Court,Venue Category_Food & Drink Shop,Venue Category_Flea Market,Venue Category_Fish & Chips Shop,...,0.0,0.0,0,Central,13.0031,77.5643,Bun World,13.007511,77.563738,Bakery
417,Malleswaram,Venue Category_Ice Cream Shop,Venue Category_Breakfast Spot,Venue Category_Indian Restaurant,Venue Category_Bakery,Venue Category_Falafel Restaurant,Venue Category_Food Court,Venue Category_Food & Drink Shop,Venue Category_Flea Market,Venue Category_Fish & Chips Shop,...,0.0,0.0,0,Central,13.0031,77.5643,Amrith Ice creams,13.001860,77.567340,Ice Cream Shop
418,Malleswaram,Venue Category_Ice Cream Shop,Venue Category_Breakfast Spot,Venue Category_Indian Restaurant,Venue Category_Bakery,Venue Category_Falafel Restaurant,Venue Category_Food Court,Venue Category_Food & Drink Shop,Venue Category_Flea Market,Venue Category_Fish & Chips Shop,...,0.0,0.0,0,Central,13.0031,77.5643,baskin robbins,13.007388,77.563982,Ice Cream Shop


In [31]:
target_cluster = target_cluster_dataframe.iloc[0].at['Cluster']
print("The target cluster is : ",target_cluster)

The target cluster is :  0


In [32]:
possible_neighborhoods = neighborhoods_venues_sorted[neighborhoods_venues_sorted['Cluster']==target_cluster]
possible_neighborhoods.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,Venue Category_Women's Store,Venue Category_Yoga Studio,Cluster,Borough,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Arekere,Venue Category_Indian Restaurant,Venue Category_Sporting Goods Shop,Venue Category_Mughlai Restaurant,Venue Category_Liquor Store,Venue Category_Bus Line,Venue Category_Burger Joint,Venue Category_Neighborhood,Venue Category_Breakfast Spot,Venue Category_Pizza Place,...,0.0,0.0,0,SouthernSuburbs,12.8875,77.597,Decathlon Sports India Pvt Ltd,12.887513,77.597712,Sporting Goods Shop
1,Arekere,Venue Category_Indian Restaurant,Venue Category_Sporting Goods Shop,Venue Category_Mughlai Restaurant,Venue Category_Liquor Store,Venue Category_Bus Line,Venue Category_Burger Joint,Venue Category_Neighborhood,Venue Category_Breakfast Spot,Venue Category_Pizza Place,...,0.0,0.0,0,SouthernSuburbs,12.8875,77.597,Zhangs Dynasty,12.889923,77.597804,Chinese Restaurant
2,Arekere,Venue Category_Indian Restaurant,Venue Category_Sporting Goods Shop,Venue Category_Mughlai Restaurant,Venue Category_Liquor Store,Venue Category_Bus Line,Venue Category_Burger Joint,Venue Category_Neighborhood,Venue Category_Breakfast Spot,Venue Category_Pizza Place,...,0.0,0.0,0,SouthernSuburbs,12.8875,77.597,Pizza Hut,12.889420,77.597494,Pizza Place
3,Arekere,Venue Category_Indian Restaurant,Venue Category_Sporting Goods Shop,Venue Category_Mughlai Restaurant,Venue Category_Liquor Store,Venue Category_Bus Line,Venue Category_Burger Joint,Venue Category_Neighborhood,Venue Category_Breakfast Spot,Venue Category_Pizza Place,...,0.0,0.0,0,SouthernSuburbs,12.8875,77.597,Guru Garden,12.891196,77.597979,Indian Restaurant
4,Arekere,Venue Category_Indian Restaurant,Venue Category_Sporting Goods Shop,Venue Category_Mughlai Restaurant,Venue Category_Liquor Store,Venue Category_Bus Line,Venue Category_Burger Joint,Venue Category_Neighborhood,Venue Category_Breakfast Spot,Venue Category_Pizza Place,...,0.0,0.0,0,SouthernSuburbs,12.8875,77.597,Kababis,12.889605,77.597557,Indian Restaurant


In [33]:
print("There are {} neighborhoods which has similar characteristics to Malleswaram.".format(possible_neighborhoods.shape[0]))

There are 387 neighborhoods which has similar characteristics to Malleswaram.


In [34]:
possible_neighborhoods.reset_index().head()

,index,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,...,Venue Category_Women's Store,Venue Category_Yoga Studio,Cluster,Borough,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,0,Arekere,Venue Category_Indian Restaurant,Venue Category_Sporting Goods Shop,Venue Category_Mughlai Restaurant,Venue Category_Liquor Store,Venue Category_Bus Line,Venue Category_Burger Joint,Venue Category_Neighborhood,Venue Category_Breakfast Spot,...,0.0,0.0,0,SouthernSuburbs,12.8875,77.597,Decathlon Sports India Pvt Ltd,12.887513,77.597712,Sporting Goods Shop
1,1,Arekere,Venue Category_Indian Restaurant,Venue Category_Sporting Goods Shop,Venue Category_Mughlai Restaurant,Venue Category_Liquor Store,Venue Category_Bus Line,Venue Category_Burger Joint,Venue Category_Neighborhood,Venue Category_Breakfast Spot,...,0.0,0.0,0,SouthernSuburbs,12.8875,77.597,Zhangs Dynasty,12.889923,77.597804,Chinese Restaurant
2,2,Arekere,Venue Category_Indian Restaurant,Venue Category_Sporting Goods Shop,Venue Category_Mughlai Restaurant,Venue Category_Liquor Store,Venue Category_Bus Line,Venue Category_Burger Joint,Venue Category_Neighborhood,Venue Category_Breakfast Spot,...,0.0,0.0,0,SouthernSuburbs,12.8875,77.597,Pizza Hut,12.889420,77.597494,Pizza Place
3,3,Arekere,Venue Category_Indian Restaurant,Venue Category_Sporting Goods Shop,Venue Category_Mughlai Restaurant,Venue Category_Liquor Store,Venue Category_Bus Line,Venue Category_Burger Joint,Venue Category_Neighborhood,Venue Category_Breakfast Spot,...,0.0,0.0,0,SouthernSuburbs,12.8875,77.597,Guru Garden,12.891196,77.597979,Indian Restaurant
4,4,Arekere,Venue Category_Indian Restaurant,Venue Category_Sporting Goods Shop,Venue Category_Mughlai Restaurant,Venue Category_Liquor Store,Venue Category_Bus Line,Venue Category_Burger Joint,Venue Category_Neighborhood,Venue Category_Breakfast Spot,...,0.0,0.0,0,SouthernSuburbs,12.8875,77.597,Kababis,12.889605,77.597557,Indian Restaurant


### Create Ranking by combining population and income factors

In [39]:
possible_neighborhoods.rename(columns={'Neighborhood':'Neighborhoods'},inplace=True)

In [36]:
possible_neighborhoods = possible_neighborhoods.merge(population[['Neighborhoods','Population','Normalized_population']],on='Neighborhoods')
possible_neighborhoods = possible_neighborhoods.merge(income[['Neighborhoods','AverageIncome','Normalized_income']],on='Neighborhoods')
possible_neighborhoods = possible_neighborhoods.merge(veg[['Neighborhoods','NumberOfVegRestaurants','NumberOfNonVegRestaurants']],on='Neighborhoods')
possible_neighborhoods.head(10)

,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,Venue,Venue Latitude,Venue Longitude,Venue Category,Population,Normalized_population,AverageIncome,Normalized_income,NumberOfVegRestaurants,NumberOfNonVegRestaurants
0,Arekere,Venue Category_Indian Restaurant,Venue Category_Sporting Goods Shop,Venue Category_Mughlai Restaurant,Venue Category_Liquor Store,Venue Category_Bus Line,Venue Category_Burger Joint,Venue Category_Neighborhood,Venue Category_Breakfast Spot,Venue Category_Pizza Place,...,Decathlon Sports India Pvt Ltd,12.887513,77.597712,Sporting Goods Shop,138760,0.141072,29378.716626,0.454466,0,1.0
1,Arekere,Venue Category_Indian Restaurant,Venue Category_Sporting Goods Shop,Venue Category_Mughlai Restaurant,Venue Category_Liquor Store,Venue Category_Bus Line,Venue Category_Burger Joint,Venue Category_Neighborhood,Venue Category_Breakfast Spot,Venue Category_Pizza Place,...,Zhangs Dynasty,12.889923,77.597804,Chinese Restaurant,138760,0.141072,29378.716626,0.454466,0,1.0
2,Arekere,Venue Category_Indian Restaurant,Venue Category_Sporting Goods Shop,Venue Category_Mughlai Restaurant,Venue Category_Liquor Store,Venue Category_Bus Line,Venue Category_Burger Joint,Venue Category_Neighborhood,Venue Category_Breakfast Spot,Venue Category_Pizza Place,...,Pizza Hut,12.889420,77.597494,Pizza Place,138760,0.141072,29378.716626,0.454466,0,1.0
3,Arekere,Venue Category_Indian Restaurant,Venue Category_Sporting Goods Shop,Venue Category_Mughlai Restaurant,Venue Category_Liquor Store,Venue Category_Bus Line,Venue Category_Burger Joint,Venue Category_Neighborhood,Venue Category_Breakfast Spot,Venue Category_Pizza Place,...,Guru Garden,12.891196,77.597979,Indian Restaurant,138760,0.141072,29378.716626,0.454466,0,1.0
4,Arekere,Venue Category_Indian Restaurant,Venue Category_Sporting Goods Shop,Venue Category_Mughlai Restaurant,Venue Category_Liquor Store,Venue Category_Bus Line,Venue Category_Burger Joint,Venue Category_Neighborhood,Venue Category_Breakfast Spot,Venue Category_Pizza Place,...,Kababis,12.889605,77.597557,Indian Restaurant,138760,0.141072,29378.716626,0.454466,0,1.0
5,Arekere,Venue Category_Indian Restaurant,Venue Category_Sporting Goods Shop,Venue Category_Mughlai Restaurant,Venue Category_Liquor Store,Venue Category_Bus Line,Venue Category_Burger Joint,Venue Category_Neighborhood,Venue Category_Breakfast Spot,Venue Category_Pizza Place,...,Madhuloka Liquor Boutique,12.890243,77.597734,Liquor Store,138760,0.141072,29378.716626,0.454466,0,1.0
6,Arekere,Venue Category_Indian Restaurant,Venue Category_Sporting Goods Shop,Venue Category_Mughlai Restaurant,Venue Category_Liquor Store,Venue Category_Bus Line,Venue Category_Burger Joint,Venue Category_Neighborhood,Venue Category_Breakfast Spot,Venue Category_Pizza Place,...,Benison Super Market,12.884668,77.594678,Department Store,138760,0.141072,29378.716626,0.454466,0,1.0
7,Arekere,Venue Category_Indian Restaurant,Venue Category_Sporting Goods Shop,Venue Category_Mughlai Restaurant,Venue Category_Liquor Store,Venue Category_Bus Line,Venue Category_Burger Joint,Venue Category_Neighborhood,Venue Category_Breakfast Spot,Venue Category_Pizza Place,...,McDonald's,12.891420,77.598551,Burger Joint,138760,0.141072,29378.716626,0.454466,0,1.0
8,Arekere,Venue Category_Indian Restaurant,Venue Category_Sporting Goods Shop,Venue Category_Mughlai Restaurant,Venue Category_Liquor Store,Venue Category_Bus Line,Venue Category_Burger Joint,Venue Category_Neighborhood,Venue Category_Breakfast Spot,Venue Category_Pizza Place,...,Dhi Sports,12.886223,77.597000,Athletics & Sports,138760,0.141072,29378.716626,0.454466,0,1.0
9,Arekere,Venue Category_Indian Restaurant,Venue Category_Sporting Goods Shop,Venue Category_Mughlai Restaurant,Venue Category_Liquor Store,Venue Category_Bus Line,Venue Category_Burger Joint,Venue Category_Neighborhoo

In [37]:
possible_neighborhoods['Ranking'] = possible_neighborhoods['Normalized_population'] * 0.5 + possible_neighborhoods['Normalized_income'] * 0.35 + possible_neighborhoods['NumberOfNonVegRestaurants'] * 0.1
recommended_neighborhoods = possible_neighborhoods.sort_values(by='Ranking',ascending=False)
recommended_neighborhoods.reset_index(inplace=True, drop=True)
recommended_neighborhoods.head(10)

,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,Venue Latitude,Venue Longitude,Venue Category,Population,Normalized_population,AverageIncome,Normalized_income,NumberOfVegRestaurants,NumberOfNonVegRestaurants,Ranking
0,Malleswaram,Venue Category_Ice Cream Shop,Venue Category_Breakfast Spot,Venue Category_Indian Restaurant,Venue Category_Bakery,Venue Category_Falafel Restaurant,Venue Category_Food Court,Venue Category_Food & Drink Shop,Venue Category_Flea Market,Venue Category_Fish & Chips Shop,...,13.006587,77.564354,Ice Cream Shop,893629,0.908516,53270.063892,0.824047,0,1.0,0.842674
1,Malleswaram,Venue Category_Ice Cream Shop,Venue Category_Breakfast Spot,Venue Category_Indian Restaurant,Venue Category_Bakery,Venue Category_Falafel Restaurant,Venue Category_Food Court,Venue Category_Food & Drink Shop,Venue Category_Flea Market,Venue Category_Fish & Chips Shop,...,13.006615,77.564158,Indian Restaurant,893629,0.908516,53270.063892,0.824047,0,1.0,0.842674
2,Malleswaram,Venue Category_Ice Cream Shop,Venue Category_Breakfast Spot,Venue Category_Indian Restaurant,Venue Category_Bakery,Venue Category_Falafel Restaurant,Venue Category_Food Court,Venue Category_Food & Drink Shop,Venue Category_Flea Market,Venue Category_Fish & Chips Shop,...,13.000799,77.563924,Breakfast Spot,893629,0.908516,53270.063892,0.824047,0,1.0,0.842674
3,Malleswaram,Venue Category_Ice Cream Shop,Venue Category_Breakfast Spot,Venue Category_Indian Restaurant,Venue Category_Bakery,Venue Category_Falafel Restaurant,Venue Category_Food Court,Venue Category_Food & Drink Shop,Venue Category_Flea Market,Venue Category_Fish & Chips Shop,...,13.001860,77.567340,Ice Cream Shop,893629,0.908516,53270.063892,0.824047,0,1.0,0.842674
4,Malleswaram,Venue Category_Ice Cream Shop,Venue Category_Breakfast Spot,Venue Category_Indian Restaurant,Venue Category_Bakery,Venue Category_Falafel Restaurant,Venue Category_Food Court,Venue Category_Food & Drink Shop,Venue Category_Flea Market,Venue Category_Fish & Chips Shop,...,13.007511,77.563738,Bakery,893629,0.908516,53270.063892,0.824047,0,1.0,0.842674
5,Malleswaram,Venue Category_Ice Cream Shop,Venue Category_Breakfast Spot,Venue Category_Indian Restaurant,Venue Category_Bakery,Venue Category_Falafel Restaurant,Venue Category_Food Court,Venue Category_Food & Drink Shop,Venue Category_Flea Market,Venue Category_Fish & Chips Shop,...,13.007388,77.563982,Ice Cream Shop,893629,0.908516,53270.063892,0.824047,0,1.0,0.842674
6,Rajajinagar,Venue Category_Bakery,Venue Category_Vegetarian / Vegan Restaurant,Venue Category_Snack Place,Venue Category_Indian Restaurant,Venue Category_Fast Food Restaurant,Venue Category_Café,Venue Category_Yoga Studio,Venue Category_Flea Market,Venue Category_Fish & Chips Shop,...,12.987441,77.549826,Bakery,981362,0.997710,60967.535874,0.943121,2,0.0,0.828947
7,Rajajinagar,Venue Category_Bakery,Venue Category_Vegetarian / Vegan Restaurant,Venue Category_Snack Place,Venue Category_Indian Restaurant,Venue Category_Fast Food Restaurant,Venue Category_Café,Venue Category_Yoga Studio,Venue Category_Flea Market,Venue Category_Fish & Chips Shop,...,12.987669,77.549978,Bakery,981362,0.997710,60967.535874,0.943121,2,0.0,0.828947
8,Rajajinagar,Venue Category_Bakery,Venue Category_Vegetarian / Vegan Restaurant,Venue Category_Snack Place,Venue Category_Indian Restaurant,Venue Category_Fast Food Restaurant,Venue Category_Café,Venue Category_Yoga Studio,Venue Category_Flea Market,Venue Category_Fish & Chips Shop,...,12.987978,77.549641,Vegetarian / Vegan Restaurant,981362,0.997710,60967.535874,0.943121,2,0.0,0.828947
9,Rajajinagar,Venue Category_Bakery,Venue Category_Vegetarian / Vegan Restaurant,Venue Category_Snack Place,Venue Category_Indian Restaurant,Venue Category_Fast Food Restaurant,Venue Category_Café,Venue Category_Yoga Studio,Venue Category_Flea Marke

In [38]:
top3 = recommended_neighborhoods.groupby(['Neighborhoods','1st Most Common Venue','2nd Most Common Venue','3rd Most Common Venue'])['Ranking'].unique()
top3_df = pd.DataFrame(top3).reset_index()
top3_df.head(3)

,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,Ranking
0,Arekere,Venue Category_Indian Restaurant,Venue Category_Sporting Goods Shop,Venue Category_Mughlai Restaurant,[0.32959888840700646]
1,BTM Layout,Venue Category_Indian Restaurant,Venue Category_Chinese Restaurant,Venue Category_Ice Cream Shop,[0.6918117751640322]
2,Banashankari,Venue Category_Indian Restaurant,Venue Category_Café,Venue Category_Breakfast Spot,[0.8234029969357849]


### Conclusion

The above results show the top 3 neighborhoods of 'Malleswaram' in terms of similar food. 

Thus, the model can recommend top 3 neighborhoods for any specified location.